In [263]:
import numpy as np
np.set_printoptions(suppress=True)
import sympy as sp
sp.init_printing(use_unicode=True)

In [279]:
c = 0.5
delta_x = 0.2
delta_t = 0.2
lam = c*delta_t/delta_x
L = lam**2

t_init = 0
t_target = 1
x_init = 0
x_target = 2

def u(x, t):
    if x == 0:
        return 0
    if t == 0:
        return x*(1-x)
    if x == l:
        return 0

def du(x, t):
    if t == 0:
        return 0
    
forcing_term = 2
  
# Currently the forcing term is simply the constant function 2. To make this implementation more general a function called 
# "forcing_term" would be defined. Then the values of the function at each node would be calculated and added to the 
# solution grid

# Example definition of forcing term function
# def forcing_term(x, t):
#     return 2
    
dims = (int(t_target/delta_t)+1, int((x_target-x_init)/delta_x)+1)

In [280]:
a = np.array([-L, 2*(1+L), -L])
A = np.eye(dims[1])
for i in range(1, dims[1]-1):
    A[i][i-1:i+2] = a
A_inv = np.linalg.inv(A)
sp.Matrix(A)

⎡ 1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0 ⎤
⎢                                                                           ⎥
⎢-0.25   2.5   -0.25   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0   -0.25   2.5   -0.25   0.0    0.0    0.0    0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0    0.0   -0.25   2.5   -0.25   0.0    0.0    0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0    0.0    0.0   -0.25   2.5   -0.25   0.0    0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0    0.0    0.0    0.0   -0.25   2.5   -0.25   0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0    0.0    0.0    0.0    0.0   -0.25   2.5   -0.25   0.0   

In [281]:
b = np.array([L/2, 1-L, L/2])
C = np.eye(dims[1])
for i in range(1, dims[1]-1):
    C[i][i-1:i+2] = b
sp.Matrix(C)

⎡ 1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0 ⎤
⎢                                                                           ⎥
⎢0.125  0.75   0.125   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0   0.125  0.75   0.125   0.0    0.0    0.0    0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0    0.0   0.125  0.75   0.125   0.0    0.0    0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0    0.0    0.0   0.125  0.75   0.125   0.0    0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0    0.0    0.0    0.0   0.125  0.75   0.125   0.0    0.0    0.0    0.0 ⎥
⎢                                                                           ⎥
⎢ 0.0    0.0    0.0    0.0    0.0   0.125  0.75   0.125   0.0   

In [282]:
grid = np.zeros((dims[0], dims[1]))

In [283]:
grid[0] = [u(x, 0) for x in delta_x*np.arange(dims[1])]
grid[0][1:dims[1]-1] = grid[0][1:dims[1]-1] + delta_x**2*forcing_term/2*np.ones((1, dims[1]-2))

In [284]:
v = np.dot(C, grid[0].T).reshape(1, dims[1]) + delta_t*np.array([[du(x, 0) for x in np.arange(dims[1])]])
v[0][1:dims[1]-1] = v[0][1:dims[1]-1] + delta_x**2*forcing_term/2*np.ones((1, dims[1]-2))
grid[1] = (4*np.dot(A_inv, v.T) - grid[0].reshape(dims[1], 1)).T
grid[1].reshape(1, dims[1])
for i in range(2, dims[0]):
    grid[i] = 4*np.dot(A_inv, grid[i-1]) - grid[i-2]
    grid[i][1:dims[1]-1] = grid[i][1:dims[1]-1] + delta_x**2*forcing_term*np.ones((1, dims[1]-2))

In [285]:
sp.Matrix(np.round(grid, decimals=4))

⎡0.0   0.2     0.28    0.28    0.2     0.04    -0.2     -0.52    -0.92      -1
⎢                                                                             
⎢0.0  0.2198  0.318   0.3198   0.24    0.08   -0.1604  -0.4843  -0.9229    -1.
⎢                                                                             
⎢0.0  0.2929  0.4119  0.4188  0.3398  0.1794  -0.065   -0.422   -1.1255    -3.
⎢                                                                             
⎢0.0  0.4083  0.5575  0.5758  0.4988  0.3356  0.0662   -0.474   -2.4544   -13.
⎢                                                                             
⎢0.0  0.5485  0.7488  0.7879  0.7143  0.5346  0.1418   -1.2229  -8.4564   -51.
⎢                                                                             
⎣0.0  0.6948  0.9776  1.0498  0.976   0.717   -0.2091  -4.9165  -32.3811  -194

.4      -2.0  ⎤
              ⎥
7843    -6.0  ⎥
              ⎥
9874    -22.0 ⎥
              ⎥
6613    -82.0 ⎥
              ⎥
55